In [98]:
# Do all imports
import pandas as pd
import scipy
import numpy as np
from numpy.f2py.auxfuncs import throw_error
from pandas import interval_range
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.pipeline import Pipeline

import os

PWD = os.getcwd()

import sys

sys.path.insert(1, PWD + './transformations.py')
sys.path.insert(1, PWD + './helpers.py')

from transformations import Winsorizer, FeatureMultiplier, Average
from helpers import logistic_map

import psycopg2

import json
import requests
import time
from http import HTTPStatus

import requests
from requests.exceptions import HTTPError


import dotenv

dotenv.load_dotenv()

True

In [99]:
APP_ID = 2
CAMPAIGN_ID = "5"
TIME_WINDOW = '10d'
FREQ = "D"  ## or "D" for daily or  "3D" 3 daily, "W-Mon"
TARGET_COUNTRY = "United_States"
VIEWS_THRESHOLD = 100

# how many % do we pull rpm's closer to its groups mean
# when rpm is under the mean
PAYOUT_UPLIFTING_PERC = 0.15

DEFAULT_RPM = 2 ## rpm we are paying to creators used as a base, currently 2$
PROCEEDS_PERCENTAGE_FOR_ATTRIBUTION = 0.8
RPM_CAP = 4  ## max rpm allowed

TARGET_ROI = 3.0

print(f"""
APP_ID: {APP_ID}
CAMPAIGN_ID: {CAMPAIGN_ID}
TIME_WINDOW: {TIME_WINDOW}
FREQ: {FREQ}
TARGET_COUNTRY: {TARGET_COUNTRY}
VIEWS_THRESHOLD: {VIEWS_THRESHOLD}

PAYOUT_UPLIFTING_PERC: {PAYOUT_UPLIFTING_PERC * 100}%

DEFAULT_RPM: {DEFAULT_RPM}$
PROCEEDS_PERCENTAGE_FOR_ATTRIBUTION: {PROCEEDS_PERCENTAGE_FOR_ATTRIBUTION * 100}%
RPM_CAP: {RPM_CAP}$

TARGET_ROI: {TARGET_ROI}
""")


APP_ID: 2
CAMPAIGN_ID: 5
TIME_WINDOW: 14d
FREQ: D
TARGET_COUNTRY: United_States
VIEWS_THRESHOLD: 100

PAYOUT_UPLIFTING_PERC: 15.0%

DEFAULT_RPM: 2$
PROCEEDS_PERCENTAGE_FOR_ATTRIBUTION: 80.0%
RPM_CAP: 4$

TARGET_ROI: 3.0



In [100]:
# define engagement factor rates
engagement_factor_weights = {
    "likes_rate":        1.0,
    "comments_rate":     1.2,
    "saves_rate":        0.5,
    "shares_rate":       0.0,
    "likes_to_comments": 1.0,
}

# engagement volume weights
engagement_volume_weights = {
    "views":    0.8,
    "likes":    1.0,
    "comments": 1.2,
    "saves":    0.3,
    "shares":   0.3,
}

# roi we are targeting for each group of views,
# posts with more views are likely to get paid more, so we cut their revenue more
# give it to posts with low views, like robin hood principal
target_roi_by_groups = [
    (-np.inf,   5_000,       1.5),
    (5_000,     10_000,      1.8),
    (10_000,    50_000,      2.3),
    (50_000,    100_000,     2.9),
    (100_000,   1_000_000,   3.0),
    (1_000_000, np.inf,      3.5),
]

# Controls how strongly marginal returns decay as volume increases; higher values imply stronger saturation effects.
#    - 1 => full effect
#    - 0 => no effect
#
# diminishing return effect by categories
# low views are less affected by diminishing returns effect as they
# don't have much room to show their effects
diminishing_returns_effect_by_group = [
    (-np.inf,   5_000,       0.2),
    (5_000,     10_000,      0.3),
    (10_000,    50_000,      0.4),
    (50_000,    100_000,     0.5),
    (100_000,   1_000_000,   0.8),
    (1_000_000, np.inf,      0.9),
]

# other configuration variables


# group data into groups by views to calculate incentive boost within each group
vol_cut_ranges = [
      -np.inf,
            0,
        1_000,
       10_000,
      100_000,
    1_000_000,
       np.inf,
]

# %-+ allowed incentive boost
incentive_boost_effect = 0.2
# how curved or linear incentive boost should be, less is more linear
incentive_boost_order = 2

## Load Data

In [101]:
query = f"""
select
    toDate(toString(person.properties.applicationInstalledAt)) as date,
    sum(properties.proceeds) as proceeds,
--     properties.$geoip_country_name as country,
    person.properties.attribution_source as attributionSource
FROM events
WHERE
    event in  ('sw_trial_converted', 'sw_subscription_start', 'sw_renewal', 'sw_refund')
    and attributionSource in ('tiktok', 'instagram') and person.properties.applicationInstalledAt is not null
--     and properties.$geoip_country_name in ('{TARGET_COUNTRY.replace("_", " ")}', 'United Kingdom')
    and "date" >= NOW() - INTERVAL '{TIME_WINDOW.rstrip('d')} days'
group by "date", attributionSource
order by "date" desc
"""

revenueRetries = 5
revenueRetryCodes = [
    HTTPStatus.TOO_MANY_REQUESTS,
    HTTPStatus.INTERNAL_SERVER_ERROR,
    HTTPStatus.BAD_GATEWAY,
    HTTPStatus.SERVICE_UNAVAILABLE,
    HTTPStatus.GATEWAY_TIMEOUT,
]

revenueResponse = None

for n in range(revenueRetries):
    try:
        revenueResponse = requests.post(
            f'https://us.posthog.com/api/projects/{276292}/query/',
            headers={
                "Authorization": f'Bearer {os.getenv("POSTHOG_KEY")}',
                'Content-Type': 'application/json',
            },
            data=json.dumps({
                'query': {
                    'kind': "HogQLQuery",
                    'query': query
                },
                'async': False,
                'refresh': 'force_blocking', # ignore cache
                'name': "Cohorted Proceeds by date, country and attribution source",
            })
        )
        revenueResponse.raise_for_status()

        break

    except HTTPError as exc:
        code = exc.response.status_code

        if code in revenueRetryCodes:
            # exponential backoff
            time.sleep(2 ** (n + 1))
            continue

        customExc = None
        try:
            customExc = Exception(f"{exc.response.json()}")
        except:
            pass

        if customExc is not None:
            raise customExc
        else:
            raise exc

if revenueResponse is None or revenueResponse.status_code != HTTPStatus.OK or revenueResponse.headers.get('Content-Type') != 'application/json':
    raise Exception(f"Unexpected revenue response code from posthog {revenueResponse.status_code}, {revenueResponse.headers.get('Content-Type')}")

revenueJson = revenueResponse.json()
revenueDataRaw = revenueJson['results']
revenueColumns = revenueJson['columns']

In [102]:
revenue_attribution_map = {
    'tiktok': 'TikTok',
    'instagram': 'Instagram'
}

df_revenue = (
    pd.DataFrame(revenueDataRaw, columns=revenueColumns)
    .assign(
        date=lambda c: pd.to_datetime(c.date, format="%Y-%m-%d"),
        # country=lambda x: x.country.map({"United_Kingdom": "United_States"}).fillna(x.country),
        channel=lambda x: x.attributionSource.map(revenue_attribution_map)
    )
    .drop(columns=['attributionSource'])
)

In [103]:
def upsert(df, filters, values):
    mask = pd.Series(True, index=df.index)
    for col, val in filters.items():
        mask &= df[col] == val

    if mask.any():
        for col, val in values.items():
            df.loc[mask, col] = df.loc[mask, col].fillna(0) + val
    else:
        new_idx = df.index.max() + 1 if len(df) > 0 else 0
        for col, val in filters.items():
            df.loc[new_idx, col] = val
        for col, val in values.items():
            df.loc[new_idx, col] = val

        # Ensure date column stays as datetime
        df['date'] = pd.to_datetime(df['date'])

    return df

In [104]:
df_revenue_updated = upsert(df_revenue, filters={ 'date': '2026-02-17', "channel": "TikTok" }, values={ 'proceeds': 15 })
df_revenue_updated = upsert(df_revenue, filters={ 'date': '2026-02-18', "channel": "TikTok" }, values={ 'proceeds': 15 })
df_revenue_updated = upsert(df_revenue, filters={ 'date': '2026-02-19', "channel": "TikTok" }, values={ 'proceeds': 15 })
df_revenue_updated = upsert(df_revenue, filters={ 'date': '2026-02-20', "channel": "TikTok" }, values={ 'proceeds': 15 })
df_revenue_updated = upsert(df_revenue, filters={ 'date': '2026-02-21', "channel": "TikTok" }, values={ 'proceeds': 15 })
df_revenue_updated = upsert(df_revenue, filters={ 'date': '2026-02-22', "channel": "TikTok" }, values={ 'proceeds': 15 })
df_revenue_updated = upsert(df_revenue, filters={ 'date': '2026-02-23', "channel": "TikTok" }, values={ 'proceeds': 15 })
df_revenue_updated

,date,proceeds,channel
0,2026-02-23,43.717087,TikTok
1,2026-02-22,69.205918,TikTok
2,2026-02-22,105.923015,Instagram
3,2026-02-21,8.490000,Instagram
4,2026-02-20,29.491249,Instagram
5,2026-02-19,54.039592,Instagram
6,2026-02-18,48.980000,TikTok
7,2026-02-18,54.039592,Instagram
8,2026-02-17,43.481715,TikTok
9,2026-02-16,27.223531,TikTok


In [105]:
stat_cols = [
    "views_diff",
    "likes_diff",
    "comments_diff",
    "saves_diff",
    "shares_diff"
]

df_scrape_data = (
    pd
    .read_sql_query(
        sql=f"""
            select
                smadd.date,
                smadd.id as account_id,
                smadd.campaign_id as campaign_id,
                smadd.country as account_country,
                smadd.channel as account_channel,
                smadd.total_views_diff as views_diff,
                smadd.total_likes_diff as likes_diff,
                smadd.total_comments_diff as comments_diff,
                smadd.total_saves_diff as saves_diff,
                smadd.total_shares_diff as shares_diff,
                smadd.total_posts_diff as posts_diff
            from social_media_accounts_daily_diff_eligible_views smadd
            where
                smadd.date >= CURRENT_DATE - INTERVAL %(TIME_WINDOW)s
                and total_views_diff > %(VIEWS_THRESHOLD)s
                and total_views_diff > smadd.total_likes_diff
                and smadd.country = %(TARGET_COUNTRY)s
                and smadd.campaign_id in ({CAMPAIGN_ID})
            order by smadd.date desc
        """,
        con=os.getenv("DATABASE_URL"),
        params={
            "TIME_WINDOW": TIME_WINDOW,
            "VIEWS_THRESHOLD": VIEWS_THRESHOLD,
            "TARGET_COUNTRY": TARGET_COUNTRY
        },
        parse_dates=['date']
    )

)
df_scrape_data[stat_cols] = df_scrape_data[stat_cols].clip(0, np.inf)

## Data prep

In [106]:
def add_aggregate_metrics(df):
    df = (
        df
        .assign(
            likes_rate=lambda x: x['likes_diff'] / x["views_diff"],
            comments_rate=lambda x: x['comments_diff'] / x["views_diff"],
            saves_rate=lambda x: x['saves_diff'] / x["views_diff"],
            shares_rate=lambda x: x['shares_diff'] / x["views_diff"],
            likes_to_comments=lambda x: (x['comments_diff'] / x["likes_diff"]).clip(0, 1),
        )
    )
    return df

In [107]:
def get_labels_for_groups(series: pd.Series, groups_with_labels):
    """
    Label a pandas Series based on value ranges.

    Parameters
    ----------
    series : pd.Series
        Numeric pandas Series to label.
    groups_with_labels : list of tuples
        Each tuple: (lower_bound, upper_bound, label)
        Bounds are inclusive on the left, exclusive on the right.

    Returns
    -------
    pd.Series
        Labeled Series.
    """
    labels = pd.Series(index=series.index)

    for low, high, label in groups_with_labels:
        mask = (series >= low) & (series < high)
        labels.loc[mask] = label

    return labels.values

In [108]:
# aggregate to get account level metrics per date
df_posts_model = (
    df_scrape_data
    .pipe(lambda d: add_aggregate_metrics(d))
    .sort_values("likes_rate")
    .dropna()
    .groupby([
        pd.Grouper(key="date", freq=FREQ, label="left", closed="left"),
        "account_id",
        "campaign_id",
        "account_country",
        "account_channel"
    ], as_index=False)
    # .resample(FREQ, on  = "date", label = "left", closed = 'left')
    .agg({v: "sum" for _, v in enumerate(stat_cols)})
    .pipe(lambda x: add_aggregate_metrics(x).fillna(0))
)

df_revenue_model = (
    df_revenue_updated
    .groupby([
        pd.Grouper(key="date", freq=FREQ, label="left", closed="left"),
        # "country",
        "channel"
    ])
    .agg({
        "proceeds": "sum"
    })
    .reset_index()
)

### Computation

#### Engagement Factor / Diminishing Returns

  - Weighted average of engagements rates (e. g. likes/views, comments/views, comments/likes etc.)
  - Because of its negative correlation with views, we can use this as a diminishing returns proxy

In [109]:
def compute_engagement_factor(rates, weights):
    pipeline = Pipeline([
        ("scaler", StandardScaler()),
        ("weights", FeatureMultiplier(weights)),
        ("average", Average()),
        ("winsor", Winsorizer(lower_quantile=0.03, upper_quantile=0.99)),  ## clip extremes
        ("minmax", MinMaxScaler(feature_range=(0, 1)))  # scale to 0-1
    ])

    return pipeline.fit_transform(rates)

engagement_factor_weights_series = pd.Series(engagement_factor_weights)

df_posts_model["engagement_factor_raw"] = compute_engagement_factor(
    rates=df_posts_model[["likes_rate", "comments_rate", "saves_rate", "shares_rate", "likes_to_comments"]].clip(0, 1),
    weights=engagement_factor_weights_series,
)

# amplitude of diminishing returns effect is decided by the views
df_posts_model["engagement_factor"] = (
        df_posts_model["engagement_factor_raw"]
        ** get_labels_for_groups(df_posts_model['views_diff'], diminishing_returns_effect_by_group)
)

In [110]:
# scatter_plot(
#     df_posts_model.query("engagement_factor > 0").assign(views_log = lambda x: np.log(x.views_diff+1)),
#     "views_log",
#     "engagement_factor",
#     xlabel = "views_log",
#     ylabel = "engagement_factor",
#     title = "Log Views vs. Engagement Factor"
# )

#### Incentive boost

- Statistical transformation on the engagement_factor
    - boxcox to make the distribution near normal
    - abs max scaling
    - stratified normalization --> to make every number comparable for ranking
    - logistic mapping for incentive multiplier
- here how it works:
    - mean performance is 1
    - the output is in 0.8 to 1.2
      - if above mean ==> max 20% boost (1.2)
      - if belov mean ==> min 20% deboost (0.8)

In [111]:
def compute_incentive_boost(df, engagement_col, group_col, boxcox_lambda=None):
    boost, _ = scipy.stats.boxcox(df[engagement_col] + 1, lmbda=boxcox_lambda)
    df = df[[engagement_col, group_col]].copy()

    df['boost'] = boost
    df['boost'] /= df['boost'].max()
    df['boost'] = df['boost'] / df.groupby(group_col, observed=False)['boost'].transform('mean')

    return df['boost'].clip(0, 2)


df_posts_model["vol_cut"] = pd.cut(df_posts_model['views_diff'], vol_cut_ranges)

df_posts_model["incentive_boost_raw"] = compute_incentive_boost(
    df=df_posts_model,
    engagement_col="engagement_factor",
    group_col="vol_cut",
)

df_posts_model['incentive_boost'] = logistic_map(df_posts_model['incentive_boost_raw'], d=incentive_boost_effect, k=incentive_boost_order)

In [112]:
# x = np.linspace(0,2,100)
# y = logistic_map(x, d=incentive_boost_effect, k=incentive_boost_order)
#
# fig, ax = plt.subplots(figsize = (12,5))
# ax.plot(x, y, ls = "", marker = 'o', markersize = 2)

In [113]:
# scatter_plot(
#     df_posts_model.query("engagement_factor > 0").assign(views_log = lambda x: np.log(x.views_diff+1)),
#     "views_log",
#     "incentive_boost_raw",
#     xlabel = "views_log",
#     ylabel = "incentive_boost",
#     title = "Log Views vs. Incentive Boost"
# )

#### Engagement Volume

  - Weighted average of volume metrics (e. g. views, shares, comments, likes)

In [114]:
def compute_engagement_volume(volumes, weights):
    pipeline = Pipeline([
        ("weights", FeatureMultiplier(weights)),
        ("average", Average()),
        ("winsor", Winsorizer(lower_quantile=0, upper_quantile=1)),
    ])

    return pipeline.fit_transform(volumes)


engagement_volume_weights_series = pd.Series(engagement_volume_weights)

df_posts_model['engagement_volume'] = compute_engagement_volume(
    volumes=(df_posts_model[stat_cols]).pipe(lambda x: x / x.quantile(0.9)).values,
    weights=engagement_volume_weights_series
)

#### Quality Volume

  - Main metric for attribution, defined as:
    - engagement_volume * engagement_factor * incentive_boost

In [115]:
df_posts_model["quality_volume"] = (
    df_posts_model["engagement_volume"] *
    df_posts_model["engagement_factor"] *
    df_posts_model["incentive_boost"]
)

#### Computing the payout

- **attr_revenue_on_view**: Attributed proceeds, if the allocation is made purely on the view percentages.
- **attr_revenue_on_quality**: Attributed proceeds, if the allocation is made on the quality volume percentage.
- **payout_raw**: Payout if a flat RPM of DEFAULT_RPM is being choosed,
- **payout_on_view**: Payout, computed on the attribted proceeds and assumed ROI, if the allocation is made purely on the view percentage.
- **payout_on_quality**: Payout, computed on the attribted proceeds and assumed ROI, if the allocation is made on the quality volume percentage.
    - payout is capped to the RPM_CAP to prevent unrealistic payouts for small creators

In [116]:
def cap_rpm(views, payout):
    rpm = payout / (views / 1000)
    return np.where(rpm < RPM_CAP, payout, RPM_CAP * (views / 1000))

In [117]:
df_payout = (
    df_posts_model
    .merge(
        df_revenue_model, left_on=["date", 'account_channel'], right_on=["date", 'channel']
    )
    .assign(
        total_views_on_day=lambda x: x.groupby(["date", "account_channel"]).views_diff.transform('sum'),
        view_percentage=lambda x: x['views_diff'] / x.groupby(["date", "account_channel"]).views_diff.transform('sum'),
        quality_percentage=lambda x: x['quality_volume'] / x.groupby(["date", "account_channel"]).quality_volume.transform('sum'),
        revenue_on_view=lambda x: x.proceeds * x["view_percentage"],
        revenue_on_quality=lambda x: x.proceeds * x["quality_percentage"],
        attr_revenue_on_view=lambda x: x.revenue_on_view * PROCEEDS_PERCENTAGE_FOR_ATTRIBUTION,
        attr_revenue_on_quality=lambda x: x.revenue_on_quality * PROCEEDS_PERCENTAGE_FOR_ATTRIBUTION,
    )
    .assign(
        payout_raw=lambda x: x.views_diff * DEFAULT_RPM / 1000,
        payout_on_view=lambda x: x.attr_revenue_on_view / get_labels_for_groups(x.views_diff, target_roi_by_groups),

        payout_on_quality_without_uplifting= lambda x: cap_rpm(x.views_diff, x.attr_revenue_on_quality / get_labels_for_groups(x.views_diff, target_roi_by_groups)),
        rpm_without_uplifting= lambda x: (x.payout_on_quality_without_uplifting / (x.views_diff / 1000)),
    )
    .assign(
        rpm_without_uplifting_mean=lambda x: x.groupby("vol_cut", observed=True).rpm_without_uplifting.transform('mean'),
        rpm=lambda x: ((x.rpm_without_uplifting_mean - x.rpm_without_uplifting) * PAYOUT_UPLIFTING_PERC * (x.rpm_without_uplifting < x.rpm_without_uplifting_mean) + x.rpm_without_uplifting).clip(0, RPM_CAP),
        payout_on_quality=lambda x: (x.views_diff / 1000) * x.rpm,
        payout_percentage=lambda x: x.payout_on_quality / x.groupby(["date", "account_channel"]).payout_on_quality.transform('sum'),
        payout_stayyou_custom_temp=lambda x: np.minimum(x.revenue_on_view / 3.0, (x.views_diff / 1000) * RPM_CAP)
    )
    .sort_values("views_diff", ascending=False)
)
# df_payout[[
#     'date',
#     'channel',
#     'account_id',
#     'proceeds',
#     'views_diff',
#     'view_percentage',
#     'payout_stayyou_custom_temp'
# ]]

## Summary

In [118]:
# filter last 3 days as they data in these days are not complete

# cutoff = now minus 3 days
three_days_before = pd.Timestamp.now() - pd.Timedelta(days=3)

# filter rows BEFORE last 3 days
df_payout_filtered = df_payout[df_payout['date'] < three_days_before]
df_revenue_filtered = df_revenue_updated[df_revenue_updated['date'] < three_days_before]

In [119]:
# Overview of the weekly data
(
    df_payout_filtered[[
        'date',
        'account_id',
        'views_diff',
        'likes_diff',
        'comments_diff',
        'saves_diff',
        'shares_diff',
        'revenue_on_view',
        'revenue_on_quality',
        'payout_on_quality_without_uplifting',
        'payout_on_quality'
    ]]
    .groupby(['account_id'])
    .resample('W-MON', on='date')
    .sum(numeric_only=True)
    .assign(
        rpm=lambda x: (x.payout_on_quality / (x.views_diff / 1000)).round(2).fillna(0),
    )
    .sort_values("views_diff", ascending=False)
)

,,account_id,views_diff,likes_diff,comments_diff,saves_diff,shares_diff,revenue_on_view,revenue_on_quality,payout_on_quality_without_uplifting,payout_on_quality,rpm
account_id,date,,,,,,,,,,,
12874,2026-02-23,64370,892606,3501,171,0,0,165.512650,26.683934,7.188503,16.427611,0.02
12282,2026-02-16,49128,314102,22897,36,7670,656,451.619589,504.243609,151.584293,151.584293,0.48
12874,2026-02-16,25748,100909,1116,56,0,0,33.281012,11.870520,3.547626,6.001165,0.06
12485,2026-02-16,24970,74042,2654,44,0,0,23.477000,23.927541,8.322623,9.264974,0.13
12729,2026-02-23,50916,67127,517,103,0,0,19.996593,31.004088,11.162516,12.229112,0.18
...,...,...,...,...,...,...,...,...,...,...,...,...
12724,2026-02-16,12724,171,7,0,1,0,0.113587,0.070281,0.037483,0.049529,0.29
13011,2026-02-16,13011,170,2,0,0,0,0.149807,0.016327,0.008708,0.024967,0.15
12975,2026-02-23,12975,137,13,0,0,0,0.063755,0.043001,0.022934,0.033650,0.25


In [120]:
# overview of the data group by vol cuts
(
    df_payout_filtered
    .query("views_diff > 0")
    .groupby("vol_cut", observed=False)
    .agg(
        {
            "account_id": "count",
            "views_diff": "sum",
            "total_views_on_day": "sum",
            "quality_volume": "sum",
            "revenue_on_view": "sum",
            "revenue_on_quality": "sum",
            "attr_revenue_on_view": "sum",
            "attr_revenue_on_quality": "sum",
            "payout_raw": "sum",
            "payout_on_view": "sum",
            "payout_on_quality_without_uplifting": "sum",
            "payout_on_quality": "sum"
        }
    )
    .assign(
        view_perc=lambda x: x.views_diff / x.views_diff.sum(),
        attr_perc_on_quality=lambda x: x.attr_revenue_on_quality / x.attr_revenue_on_quality.sum(),
        attr_perc_on_view=lambda x: x.attr_revenue_on_view / x.attr_revenue_on_view.sum(),
        rpm_without_uplifting=lambda x: x.payout_on_quality_without_uplifting / (x.views_diff / 1000),
        rpm=lambda x: x.payout_on_quality / (x.views_diff / 1000),

    )
)[[
    'account_id',
    'views_diff',
    'revenue_on_view',
    'payout_on_quality_without_uplifting',
    'rpm_without_uplifting',
    'payout_on_quality',
    'rpm'
]]

,account_id,views_diff,revenue_on_view,payout_on_quality_without_uplifting,rpm_without_uplifting,payout_on_quality,rpm
vol_cut,,,,,,,
"(-inf, 0.0]",0,0,0.000000,0.000000,NaN,0.000000,NaN
"(0.0, 1000.0]",220,87554,118.244317,68.204903,0.779004,72.257888,0.825295
"(1000.0, 10000.0]",135,371690,277.707835,176.733570,0.475486,191.021792,0.513928
"(10000.0, 100000.0]",27,748377,518.737559,162.765452,0.217491,176.196060,0.235438
"(100000.0, 1000000.0]",4,921159,242.169334,39.818376,0.043226,44.349172,0.048145
"(1000000.0, inf]",0,0,0.000000,0.000000,NaN,0.000000,NaN


In [121]:
# final rpm
virality_perc = 1.25
results = {
    "total_payout_without_uplifting": df_payout_filtered['payout_on_quality_without_uplifting'].sum().round(1),
    "total_payout": df_payout_filtered['payout_on_quality'].sum().round(1),
    "total_payout_with_virality_costs": (df_payout_filtered['payout_on_quality'].sum() * virality_perc).round(1),
    "total_revenue": df_revenue_filtered['proceeds'].sum().round(1),
    "rpm_before_virality_costs": (df_revenue_filtered['proceeds'].sum() / (df_payout_filtered['payout_on_quality'].sum())).round(2),
    "overall_rpm": (df_revenue_filtered['proceeds'].sum() / (df_payout_filtered['payout_on_quality'].sum() * virality_perc)).round(2)
}
for k, v in results.items():
    print("{:<35} {:>10}".format(k, v))

total_payout_without_uplifting           447.5
total_payout                             483.8
total_payout_with_virality_costs         604.8
total_revenue                           1156.9
rpm_before_virality_costs                 2.39
overall_rpm                               1.91


## Upsert Data to Postgres

In [122]:
# PREPARE INSERTION DATA

# we store always the last two weeks although window is always 30 days
# in order
two_week_ago = pd.Timestamp.now() - pd.Timedelta(days=14)
df_payout_last_two_weeks = df_payout[df_payout['date'] >= two_week_ago]

# MAKE DATA CONTINUOUS BY DATE
# filling missing entries because no views were generated on that day
# however some videos do not pass this threshold and therefore removed from datapoints
# we need to fill these gaps to make sure data is continuous
group_cols = ['account_id', 'campaign_id', 'account_country', 'account_channel']

df_payout_last_two_weeks_continuous = (
    df_payout_last_two_weeks.groupby(group_cols)['date']
      .apply(lambda s: pd.date_range(s.min(), s.max(), freq="D"))
      .explode()
      .rename("date")
      .reset_index()
)

zero_cols = list(set(df_payout_last_two_weeks.columns.values) - set(group_cols + ['vol_cut', 'date']))
default_values_dict = dict.fromkeys(zero_cols, 0)
default_values_dict['vol_cut'] = pd.Interval(left=-np.inf, right=0, closed='right')

df_payout_last_two_weeks = (
    df_payout_last_two_weeks_continuous.merge(
        df_payout_last_two_weeks,
        on=group_cols + ['date'],
        how='left',
    )
    .fillna(default_values_dict)
    .assign(
        date_str=lambda x: x.date.dt.strftime("%Y-%m-%d"),
        vol_cut_str=lambda x: x.vol_cut.map(lambda y: str(y)),
        # TODO: changed temporarily for custom rpm calculation
        payout_on_quality=lambda x: x.payout_stayyou_custom_temp,
        rpm=lambda x: (x.payout_stayyou_custom_temp / (x.views_diff / 1000)).fillna(0)
    )
)
df_payout_last_two_weeks

,account_id,campaign_id,account_country,account_channel,date,views_diff,likes_diff,comments_diff,saves_diff,shares_diff,...,payout_on_view,payout_on_quality_without_uplifting,rpm_without_uplifting,rpm_without_uplifting_mean,rpm,payout_on_quality,payout_percentage,payout_stayyou_custom_temp,date_str,vol_cut_str
0,12282,5,United_States,TikTok,2026-02-13,198753.0,11910.0,21.0,3809.0,331.0,...,35.205771,34.813494,0.175160,0.048740,0.221417,44.007214,0.815116,44.007214,2026-02-13,"(100000.0, 1000000.0]"
1,12282,5,United_States,TikTok,2026-02-14,35421.0,3217.0,3.0,1181.0,110.0,...,45.593134,53.914085,1.522094,0.197252,1.233546,43.693420,0.897720,43.693420,2026-02-14,"(10000.0, 100000.0]"
2,12282,5,United_States,TikTok,2026-02-15,72204.0,7074.0,10.0,2429.0,193.0,...,50.127212,56.190840,0.778223,0.197252,0.838878,60.570381,0.922664,60.570381,2026-02-15,"(10000.0, 100000.0]"
3,12282,5,United_States,TikTok,2026-02-16,7724.0,696.0,2.0,251.0,22.0,...,3.025131,6.665874,0.863008,0.535147,0.293740,2.268848,0.471691,2.268848,2026-02-16,"(1000.0, 10000.0]"
4,12282,5,United_States,TikTok,2026-02-17,2050.0,89.0,3.0,28.0,4.0,...,2.341872,3.296949,1.608268,0.535147,0.713985,1.463670,0.141653,1.463670,2026-02-17,"(1000.0, 10000.0]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
476,13155,5,United_States,TikTok,2026-02-21,1015.0,19.0,0.0,5.0,0.0,...,0.335940,0.069162,0.068139,0.535147,0.206859,0.209962,0.015661,0.209962,2026-02-21,"(1000.0, 10000.0]"
477,13155,5,United_States,TikTok,2026-02-22,365.0,12.0,0.0,1.0,1.0,...,0.584904,0.330691,0.906004,0.688840,1.001547,0.365565,0.009212,0.365565,2026-02-22,"(0.0, 1000.0]"
478,13155,5,United_States,TikTok,2026-02-23,592.0,15.0,0.0,6.0,0.0,...,0.773101,0.218097,0.368407,0.688840,0.816196,0.483188,0.011792,0.483188,2026-02-23,"(0.0, 1000.0]"
479,13175,5,United_States,TikTok,2026-02-22,457.0,1.0,0.0,0.0,0.0,...,0.732331,0.000000,0.000000,0.688840,1.001547,0.457707,0.001315,0.457707,2026-02-22,"(0.0, 1000.0]"


In [123]:
INSERT_BATCH_SIZE = 100

payout_insert_cols = {
    'date_str': 'date',
    'account_id': 'account_id',
    'campaign_id': 'campaign_id',
    'account_country': 'account_country',
    'views_diff': 'views_diff',
    'likes_diff': 'likes_diff',
    'comments_diff': 'comments_diff',
    'saves_diff': 'saves_diff',
    'shares_diff': 'shares_diff',
    'likes_rate': 'likes_rate',
    'comments_rate': 'comments_rate',
    'saves_rate': 'saves_rate',
    'shares_rate': 'shares_rate',
    'likes_to_comments': 'likes_to_comments',
    'engagement_factor_raw': 'engagement_factor_raw',
    'engagement_factor': 'engagement_factor',
    'vol_cut_str': 'vol_cut',
    'incentive_boost_raw': 'incentive_boost_raw',
    'incentive_boost': 'incentive_boost',
    'engagement_volume': 'engagement_volume',
    'quality_volume': 'quality_volume',
    'proceeds': 'proceeds',
    'total_views_on_day': 'total_views_on_day',
    'view_percentage': 'view_percentage',
    'quality_percentage': 'quality_percentage',
    'revenue_on_view': 'revenue_on_view',
    'revenue_on_quality': 'revenue_on_quality',
    'attr_revenue_on_view': 'attr_revenue_on_view',
    'attr_revenue_on_quality': 'attr_revenue_on_quality',
    'payout_raw': 'payout_raw',
    'payout_on_view': 'payout_on_view',
    'payout_on_quality_without_uplifting': 'payout_on_quality_without_uplifting',
    'rpm_without_uplifting': 'rpm_without_uplifting',
    'rpm_without_uplifting_mean': 'rpm_without_uplifting_mean',
    'rpm': 'rpm',
    'payout_on_quality': 'payout_on_quality',
    'payout_percentage': 'payout_percentage',
}

upsert_ignore_list = ['date', 'account_id', 'campaign_id']
upsert_keys = filter(lambda x: x not in upsert_ignore_list, payout_insert_cols.values())

df_payout_last_two_weeks_reduced = df_payout_last_two_weeks[payout_insert_cols.keys()]

data = list(df_payout_last_two_weeks_reduced.itertuples(index=False, name=None))
UPSERT_COMMA_SEPERATOR = ",\n"

with psycopg2.connect(os.getenv("DATABASE_URL")) as conn:
    with conn.cursor() as cur:
        insert_query = f"""
            insert into public.account_daily_rpm_calculations ({", ".join(payout_insert_cols.values())}) values %s
            on conflict on constraint account_daily_rpm_calculations_pk do update
            set
                {UPSERT_COMMA_SEPERATOR.join(list(map(lambda x: f"{x} = excluded.{x}", upsert_keys)))},
                updated_at = NOW()
        """
        psycopg2.extras.execute_values (
            cur, insert_query, data, template=None, page_size=INSERT_BATCH_SIZE
        )
    conn.commit()
